In [1]:
import os, sys
import numpy as np
import gdal


def load_band(fname):
    
    #open input image
    input_ds = gdal.Open(fname, gdal.GA_ReadOnly)
    
    print (input_ds.RasterCount)
    
    if input_ds is None:
        print ("Could not open " + fname)
        sys.exit(1)

    input_array = np.array(input_ds.GetRasterBand(1).ReadAsArray()).astype(np.float64)

    cols = input_ds.RasterXSize
    rows = input_ds.RasterYSize
    geoT = input_ds.GetGeoTransform()
    proJ = input_ds.GetProjection()
    originX = geoT[0]
    originY = geoT[3]
    pixelWidth = geoT[1]
    pixelHeight = geoT[5]

    return cols, rows, geoT, proJ, originX, originY, pixelWidth, pixelHeight, input_array

def create_output_file(infile, product_output_name, output_array):
    """
    import header information from original band file
    """
    cols, rows, geoT, proJ  = load_band(infile)[0:4]
    driver = gdal.GetDriverByName("GTiff")
    out_array = driver.Create(product_output_name, cols, rows, 1, gdal.GDT_Float32)
    out_array.SetGeoTransform(geoT)
    out_array.SetProjection(proJ)
    out_array.GetRasterBand(1).WriteArray(output_array)
    out_array.GetRasterBand(1).WriteArray(output_array.astype(np.float32))
    out_array = None
    

def create_outdir_and_outname(filepath, product):
    """
      
    Parameters
    filepath: string
        absolute path to the file
    
    product: str
        define product; ndvi, ndwi or evi
    
    return
    
    pixel_coord: tuple
        pixel coordinate that corresponds to row, column format
    #filepath = '/Users/hkim/Projects/Modis_Calibrator_Dev/ModisCalibrator_testdata/ \
    #    index_cross_cal_test_data/mrc_test_red.tif'
    #product = 'ndvi' or 'evi' or 'ndwi
    """
    
    input_dir = os.path.dirname(filepath)
    band_name = os.path.basename(filepath)
    print (band_name)
    output_dir = os.path.join(input_dir, product)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print ('Creating dir: ', output_dir)

    product_output_name = os.path.splitext(band_name)[0].split('_')[0] + '_' + product + '.tif'

    return output_dir, product_output_name


def perform_rotation(infile, angle):
    """
    # performa rotation using the angle, if you want to rotate the image around the center 
    # of the origin (originX, originY), add "cellSizeX / 2" to GeoTransform[0], and subtract
    # from GeoTransform[3]
    """
    cols, rows, geoT, proJ, originX, originY, pixelWidth, pixelHeight, array = load_band(infile)
    
    rotation = np.radians(angle)
    
    GeoTransform = list(geoT)
    GeoTransform[0] = originX;
    GeoTransform[1] = np.cos(rotation) * pixelWidth;
    GeoTransform[2] = -np.sin(rotation) * pixelWidth;
    GeoTransform[3] = originY;
    GeoTransform[4] = np.sin(rotation) * pixelHeight;
    GeoTransform[5] = np.cos(rotation) * pixelHeight;

    return tuple(GeoTransform)

In [2]:
data_dir = '/data'
os.chdir(data_dir)

infile = 'sample_s2_ndvi.tif'
outfile = 'sample_s2_ndvi_rotated.tif'

cols, rows, geoT, proJ, originX, originY, pixelWidth, pixelHeight, array = load_band(infile)

post_geoT = perform_rotation(infile, 30)

driver = gdal.GetDriverByName("GTiff")
out_array = driver.Create(outfile, cols, rows, 1, gdal.GDT_Float32)
out_array.SetGeoTransform(post_geoT)
out_array.SetProjection(proJ)
out_array.GetRasterBand(1).WriteArray(array)
out_array.GetRasterBand(1).WriteArray(array.astype(np.float32))
out_array = None


1
1
